In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import contextily as cx
import plotly.graph_objects as go
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import matplotlib
import subprocess
import torch
import joblib
import calflops
import time 

from sklearn.metrics import mean_squared_error, confusion_matrix, auc, f1_score, matthews_corrcoef
from calflops.flops_counter import calculate_flops
from shapely.geometry import MultiPoint
from sklearn.cluster import KMeans
from tsmoothie import LowessSmoother, ExponentialSmoother
from pyprojroot import here
from scipy.spatial import ConvexHull
from torch.utils.data import DataLoader, TensorDataset

import source.nn.models as models
import source.utils.utils as utils
import source.utils.fault_detection as fd

from source.utils.utils import roc_params, compute_auc, get_auc, best_mcc, best_f1score, otsuThresholding
from source.utils.utils import synthetic_timeseries
from source.utils.utils import plotly_signal

from importlib import reload
models = reload(models)
utils = reload(utils)
fd = reload(fd)

from pyprojroot import here
root_dir = str(here())

insar_dir = os.path.expanduser('~/data/raw/')
data_path = root_dir + '/data/interim/'
dataset_path = root_dir + "/data/datasets/"

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'DejaVu Serif'})

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [ ]:
dataset_name = 'EGMS_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Optuna_analysis/model_dict_{dataset_name}.pkl')

model = 'GCNAE'

print(model_dict[model]['trial_params'])
print(model_dict[model]['auc'])

In [ ]:
model_dict['GCNAE'].keys()

In [ ]:
model_dict[model]['model']

In [ ]:
np.mean(model_dict['GCNAE']['auc_evolution'],axis=0)

In [ ]:
np.mean(model_dict['GCNAE']['auc_evolution'], axis=0)

In [ ]:
dataset_name = 'EGMS_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Testing/model_dict_testing_{dataset_name}.pkl')

model_names = ['AE', 'GCN2MLP', 'GCNAE', 'GConv2MLP', 'GConvAE', 'GUNet', 'RAE_GRU', 'RAE_LSTM']

# Dictionary to store metrics for each model
metrics_dict = {}

for model_name in model_names:
    print(f"Computing metrics for {model_name}")
    auc_list = []
    f1_list = []
    mcc_list = []
    
    for idx, dataset in enumerate(datasets):
        # Compute metrics for each dataset based on each label being true if any anomaly is present
        label = dataset['label'].any(axis=1)
        scores = model_dict[model_name]['scores'][idx]
        
        auc = get_auc(scores, label, resolution=101).round(3)
        f1 = best_f1score(scores, label).round(3)
        mcc = best_mcc(scores, label).round(3)
        
        auc_list.append(auc)
        f1_list.append(f1)
        mcc_list.append(mcc)
    
    # Store metrics and compute statistics
    metrics_dict[model_name] = {
        'aucs': auc_list,
        'f1s': f1_list,
        'mccs': mcc_list,
        'mean_auc': np.mean(auc_list).round(3),
        'mean_f1': np.mean(f1_list).round(3),
        'mean_mcc': np.mean(mcc_list).round(3),
    }

In [ ]:
for model_name in model_names:
    auc_list = metrics_dict[model_name]['aucs']
    mean_auc = metrics_dict[model_name]['mean_auc']
    mean_f1 = metrics_dict[model_name]['mean_f1']
    mean_mcc = metrics_dict[model_name]['mean_mcc']
    
    # Calculate means for Malmo (first 72) and Oslo (last 72)
    malmo_mean = np.mean(auc_list[:72]).round(3)
    oslo_mean = np.mean(auc_list[-72:]).round(3)
    
    print(f"{model_name}:")
    print(f"  Training Mean: {model_dict[model_name]['auc']}")
    print(f"  Test AUC: {mean_auc}")
    print(f"  Test F1: {mean_f1}")
    print(f"  Test MCC: {mean_mcc}")
    print(f"  Test (Malmö): {malmo_mean}")
    print(f"  Test (Oslo): {oslo_mean}\n")

In [ ]:
def label_accuracy(metric, labels, interp=True):
    label = labels.any(axis=1)
    
    thr_list = list(np.linspace(metric.min(), metric.max(),101))

    f1score = []
    for threshold in thr_list[0:-1]:
        y = (metric>threshold)
        f1score.append(f1_score(label, y))

    label_1 = (labels==1).any(axis=1)
    label_2 = (labels==2).any(axis=1)

    thr_max = thr_list[np.argmax(f1score)]
    detections = (metric>thr_max)
    
    # Calculate accuracy for label_1
    true_positives = np.sum(detections & label_1)
    total_label_1 = np.sum(label_1)
    accuracy_label_1 = true_positives / total_label_1 if total_label_1 > 0 else 0

    # Calculate accuracy for label_2
    true_positives = np.sum(detections & label_2)
    total_label_2 = np.sum(label_2)
    accuracy_label_2 = true_positives / total_label_2 if total_label_2 > 0 else 0   
    
    return accuracy_label_1, accuracy_label_2

In [ ]:
dataset_name = 'Geological_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Testing/model_dict_testing_{dataset_name}.pkl')

model_names = ['AE', 'GCN2MLP', 'GCNAE', 'GConv2MLP', 'GConvAE', 'GUNet', 'RAE_GRU', 'RAE_LSTM']

# Dictionary to store metrics for each model
metrics_dict = {}

for model_name in model_names:
    print(f"Computing metrics for {model_name}")
    acc1_list = []
    acc2_list = []
    
    for idx, dataset in enumerate(datasets):
        # Compute metrics for each dataset based on each label being true if any anomaly is present
        scores = model_dict[model_name]['scores'][idx]
        
        acc1, acc2 = label_accuracy(scores, dataset['label'])
        acc1_list.append(acc1)
        acc2_list.append(acc2)
    
    # Store metrics and compute statistics
    metrics_dict[model_name] = {
        'acc1s': acc1_list,
        'acc2s': acc2_list,
        'mean_acc1': np.mean(acc1_list).round(3),
        'mean_acc2': np.mean(acc2_list).round(3),
    }

In [ ]:
for model_name in model_names:
    print(f"{model_name:<10}: - Geo: {metrics_dict[model_name]['mean_acc1']:<5}, Phase: {metrics_dict[model_name]['mean_acc2']:<5}")


In [ ]:
dataset_name = 'EGMS_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Testing/model_dict_testing_{dataset_name}.pkl')

dataset = datasets[0]
labels = dataset['label']

In [ ]:
(labels==1).any(axis=1)

-----------------------------

In [ ]:
len(model_dict['AE']['scores'])

In [ ]:
px.line(datasets[0]['label'].max(axis=1))

In [ ]:
datasets[0]['data'].shape

In [ ]:
model_orig = copy.deepcopy(model_dict['AE']['model'])

# Modify input layer size
model_orig.encoder[0].in_features = 300  # new input size
model_orig.encoder[0].weight = torch.nn.Parameter(torch.randn(25, 300))  # new weight matrix
model_orig.encoder[0].bias = torch.nn.Parameter(torch.randn(25))  # new bias vector

# Modify output layer size
model_orig.decoder[-1].out_features = 300  # new output size
model_orig.decoder[-1].weight = torch.nn.Parameter(torch.randn(300, 25))  # new weight matrix
model_orig.decoder[-1].bias = torch.nn.Parameter(torch.randn(300))  # new bias vector

In [ ]:
model_name = 'RAE_GRU'

In [ ]:
model_name.split('_')[-1].lower()

In [ ]:
model_dict['GUNet']['trial_params']

In [ ]:
[150] + [model_dict['AE']['trial_params'][f'layer_dim_{i}'] for i in range(model_dict['AE']['trial_params']['n_layers'])]


In [ ]:
relevant_params = ['n_features', 'latent_dim', 'rnn_type', 'rnn_act', 'device']
new_model_params = {key: getattr(model, key) for key in relevant_params}
new_model_params['n_features'] = X.shape[0]
model = models.RAE(**new_model_params)
model.to(new_model_params['device'])

In [ ]:
model_dict['AE']['model'].decoder[-1].out_features

### Number of parameters

In [ ]:
dataset_name = 'EGMS_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Training/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Optuna_analysis/model_dict_{dataset_name}.pkl')
model_dict['AE'].keys()

model_names = ['AE', 'GCN2MLP', 'GCNAE', 'GConv2MLP', 'GConvAE', 'GUNet', 'RAE_GRU', 'RAE_LSTM']

bar_width = 0.75

# Base x locations
x = np.arange(len(model_names))
x_positions = 2*(x)

# Extract num_parameters and total_params from model_dict
num_parameters = [model_dict[model]['num_parameters'] for model in model_names]
total_params = [model_dict[model]['num_parameters'] * model_dict[model]['trial_params']['n_epochs'] for model in model_names]


# Create the bar chart
fig = go.Figure()

# Add bars for num_parameters
fig.add_trace(go.Bar(
    x=x_positions - bar_width / 2,
    y=num_parameters,
    width=bar_width,
    name='Count of Trainable Parameters',
    text=num_parameters,
    textposition='outside'
))

# Add bars for total_params
fig.add_trace(go.Bar(
    x=x_positions + bar_width / 2,
    y=total_params,
    width=bar_width,
    name='Count of Parameter Updates',
    text=total_params,
    textposition='outside'
))

# Update layout
fig.update_layout(
    xaxis=dict(
        tickvals=x_positions,
        ticktext=[m.replace('_','') for m in model_names],
    ),
    yaxis=dict(
        tickformat="~s",
        tickvals=[i*1000 for i in range(0, 1001, 100)],  # Example: 0k, 100k, 200k, ..., 1000k
    ),
    barmode='group',
    width=1000,
    height=600,
    legend=dict(
        x=0.02,  # Horizontal position of the legend
        y=0.95,  # Vertical position of the legend
        bgcolor='rgba(255,255,255,0.5)',  # Semi-transparent background
        bordercolor='black',
        borderwidth=1,
    ),
    font=dict(
        family="Times New Roman, Times, serif",
        size=18
    )
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.write_image(root_dir + f'/outputs/figs/complexity_parameters_EGMS.pdf')
fig.show()

In [ ]:
dataset_name = 'Geological_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Training/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Optuna_analysis/model_dict_{dataset_name}.pkl')

model_names = ['AE', 'GCN2MLP', 'GCNAE', 'GConv2MLP', 'GConvAE', 'GUNet', 'RAE_GRU', 'RAE_LSTM']

bar_width = 0.75

# Base x locations
x = np.arange(len(model_names))
x_positions = 2*(x)

# Extract num_parameters and total_params from model_dict
num_parameters = [model_dict[model]['num_parameters'] for model in model_names]
total_params = [model_dict[model]['num_parameters'] * model_dict[model]['trial_params']['n_epochs'] for model in model_names]


# Create the bar chart
fig = go.Figure()

# Add bars for num_parameters
fig.add_trace(go.Bar(
    x=x_positions - bar_width / 2,
    y=num_parameters,
    width=bar_width,
    name='Count of Trainable Parameters',
    text=num_parameters,
    textposition='outside'
))

# Add bars for total_params
fig.add_trace(go.Bar(
    x=x_positions + bar_width / 2,
    y=total_params,
    width=bar_width,
    name='Count of Parameter Updates',
    text=total_params,
    textposition='outside'
))

# Update layout
fig.update_layout(
    xaxis=dict(
        tickvals=x_positions,
        ticktext=[m.replace('_','') for m in model_names],
    ),
    yaxis=dict(
        tickformat="~s",
        tickvals=[i*1000 for i in range(0, 1001, 100)],  # Example: 0k, 100k, 200k, ..., 1000k
    ),
    barmode='group',
    width=1000,
    height=600,
    legend=dict(
        x=0.02,  # Horizontal position of the legend
        y=0.95,  # Vertical position of the legend
        bgcolor='rgba(255,255,255,0.5)',  # Semi-transparent background
        bordercolor='black',
        borderwidth=1,
    ),
    font=dict(
        family="Times New Roman, Times, serif",
        size=18
    )
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.write_image(root_dir + f'/outputs/figs/complexity_parameters_Geological.pdf')
fig.show()

### Processing Time

RUN Epoch_analysis.py

In [ ]:
dataset_name = 'EGMS_anomaly'
datasets = torch.load(dataset_path + f'{dataset_name}/Training/dataset.pt')
model_dict = torch.load(root_dir + f'/outputs/Optuna_analysis/model_dict_times_{dataset_name}.pkl')

model_names = ['AE', 'GCN2MLP', 'GCNAE', 'GConv2MLP', 'GConvAE', 'GUNet', 'RAE_GRU', 'RAE_LSTM']

In [ ]:
var = 'time_epoch'

mean_times = {model: np.mean(np.mean(model_dict[model][var], axis=1)) for model in model_names}
std_times = {model: np.std(np.mean(model_dict[model][var], axis=1)) for model in model_names}

for model in model_names:
    print(f"{model}:")
    print(f"  Epoch time: {mean_times[model]:.3f} +- {std_times[model]:.3f} seconds")

In [ ]:
var = 'time_total'

mean_times = {model: np.mean(np.mean(model_dict[model][var], axis=1)) for model in model_names}
std_times = {model: np.std(np.mean(model_dict[model][var], axis=1)) for model in model_names}

for model in model_names:
    print(f"{model}:")
    print(f"  Total time: {mean_times[model]:.3f} +- {std_times[model]:.3f} seconds")

In [ ]:
next(model_dict['AE']['model'].parameters())

In [ ]:
np.array(model_dict['AE']['time_epoch'])

In [ ]:
for model_name, model_info in model_dict.items():
    print(f"{model_name}: {model_info['trial_params']['n_epochs']} epochs")

In [ ]:
device = 'cuda'
dataset_path = root_dir + "/data/datasets/"
datafile = 'EGMS_anomaly/Training/dataset.pt'
datasets = torch.load(dataset_path + datafile)

dataset = datasets[0]
input_dim = datasets[0]['data'].shape[1]

data = dataset['data']
label = dataset['label'].max(axis=1) #label per pixel

X = torch.tensor(data).float().to( device )
X.shape[0]

In [ ]:
len(datasets)

In [ ]:
model_params = {'n_features': 2,
                'latent_dim': 4,
                'rnn_type': 'GRU',
                'rnn_act': 'relu',
                'device': device}
batch_size = 512

model_class = getattr(models, 'RAE')
model = model_class(**model_params)
model = model.to(device)


if isinstance(model, models.RAE) and (model.n_features != 1):
    relevant_params = ['n_features', 'latent_dim', 'rnn_type', 'rnn_act', 'device']
    new_model_params = {key: getattr(model, key) for key in relevant_params}
    new_model_params['n_features'] = X.shape[0]
    model = models.RAE(**new_model_params)
    model.to(new_model_params['device'])

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}\n")   


In [ ]:
model_params = {'layer_dims':[input_dim, 4, 4]}

model_class = getattr(models, 'GConvAE')
model = model_class(**model_params)
model = model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}")    

In [ ]:
model

In [ ]:
model_params = {'layer_dims':[input_dim, 4, 4]}

model_class = getattr(models, 'GCNAE')
model = model_class(**model_params)
model = model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}")    

In [ ]:
model_params = {'in_channels': input_dim,
                'out_channels': input_dim,
                'hidden_channels': 300,
                'depth': 1,
                'pool_ratios': 0.7}

model_class = getattr(models, 'GUNet')
model = model_class(**model_params)
model = model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}")  

In [ ]:
def pixel_mse(output,X):
    point_mse = torch.nn.MSELoss(reduction='none')
    return torch.mean(point_mse(output,X), axis=1)


device = 'cuda:2'
def train_model(model, X, label, lr, G=None):

    rng_seed = 0
    torch.manual_seed(rng_seed)
    torch.cuda.manual_seed(rng_seed)
    np.random.seed(rng_seed)

    loss_epoch = []
    auc_epoch = []
    scores_epoch = []

    if G is not None:
        A = torch.tensor(G.W.toarray()).float()
        A = A.to(device)    

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()
    def pixel_mse(output,X):
        point_mse = torch.nn.MSELoss(reduction='none')
        return torch.mean(point_mse(output,X), axis=1)

    model.train()
    model.reset_parameters()

    # for epoch in range(1, 1+np.max(epochs_list)):
    for epoch in range(1,1000):

        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, X)
        loss.backward()
        optimizer.step()

        

        if epoch in np.ceil(np.geomspace(1,1000,10)):


            loss_epoch.append(loss.item())

            scores = pixel_mse(output, X).detach().cpu().numpy()
            # scores_epoch.append(scores)

            auc = get_auc(scores, label, resolution=101).round(3)
            auc_epoch.append(auc)


        # if epoch in epochs_list:
        #     S_partials.append(S)...

    return auc_epoch, loss_epoch

def evaluate_model(model, datasets, lr):

    auc_epoch_list = []
    loss_epoch_list = []

    it = 0
    for dataset in datasets[:5]:

        print(f'Evaluating dataset {it}', flush=True)
        it+=1

        data = dataset['data']
        label = dataset['label'].max(axis=1) #label per pixel
        
        X = torch.tensor(data).float().to(device)

        auc, loss = train_model(model, X, label, lr)
        auc_epoch_list.append(auc)
        loss_epoch_list.append(loss)

        # auc_list.append(get_auc(scores, label).round(3))
        # f1_list.append(best_f1score(scores, label).round(3))
        # mcc_list.append(best_mcc(scores, label).round(3))

    return np.mean(auc_epoch_list, axis=0).round(3), np.mean(loss_epoch_list, axis=0).round(3)

-----------------

In [ ]:
X.shape

In [ ]:
X2 = X.clone()

X2 = X2.view(-1, X.shape[1], 1)

dataset = TensorDataset(X2, X2)  # we want to reconstruct the same input
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

# Create an iterator
data_iter = iter(dataloader)

# Get the first batch
batch_X, batch_y = next(data_iter)

if model.n_features>1:
    batch_X2 = batch_X.T.unsqueeze(0)

print(batch_X.shape)
print(batch_X2.shape)


In [ ]:
batch_X.unsqueeze(2).shape

In [ ]:
batch_X.T.unsqueeze(0).shape

In [ ]:
model_params = {'n_features': 2,
                'latent_dim': 4,
                'rnn_type': 'LSTM',
                'rnn_act': 'relu',
                'device': device}
batch_size = 512

model_class = getattr(models, 'RAE')
model = model_class(**model_params)
model = model.to(device)

In [ ]:
relevant_params = ['n_features', 'latent_dim', 'rnn_type', 'rnn_act', 'device']
model_params = {key: getattr(model, key) for key in relevant_params if hasattr(model, key)}


In [ ]:
asd = 'all'

In [ ]:
asd != 1

In [ ]:
n_features = 1
batch_size = 27
seq_len = 10

x = torch.tensor([])
for i in range(seq_len):
    x_i = i*torch.ones([batch_size, n_features])

    if x_i.dim() == 1:
        x = torch.cat([x, x_i.unsqueeze(0)], axis=1)
    else:
        x = torch.cat([x, x_i], axis=1)        

In [ ]:
x.view(-1, seq_len, n_features).shape

In [ ]:
X

In [ ]:
new_params

In [ ]:
new_params['n_features'] = 300

---------

In [ ]:
study = joblib.load(root_dir+'/outputs/pixel_detection/HP_training/TR_AE.pkl')
datasets = torch.load(dataset_path + 'Oslo/training/dataset.pt')
input_dim = datasets[0]['data'].shape[1]

dataset = datasets[9]
data = dataset['data']
label = dataset['label'].max(axis=1) #label per pixel
X = torch.tensor(data).float().to(device)

px.imshow(dataset['label'], aspect='auto', width=600, title=f'Example: {label.sum():.3g} anomalous nodes').show()

In [ ]:
dataset = datasets[9]
print(dataset['metadata'])
data = dataset['data']
label = dataset['label'].max(axis=1) #label per pixel
X = torch.tensor(data).float().to(device)

lr = study.best_params['lr']
n_epochs = study.best_params['n_epochs']
n_layers = study.best_params['n_layers']
layer_dims = [input_dim]
for i in range(n_layers):
    layer_dims.append(study.best_params[f'layer_dim_{i}'])

# dims = [177, 89, 49, 35, 17]
# layer_dims = [input_dim, *dims]
# lr = 0.000025	
# n_epochs = 261

model = models.AE(layer_dims)
model = model.to(device)

rng_seed = 0
torch.manual_seed(rng_seed)
torch.cuda.manual_seed(rng_seed)
np.random.seed(rng_seed)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.MSELoss()

model.train()
model.reset_parameters()

output_list = []

# for epoch in range(1, 1+np.max(epochs_list)):
for epoch in range(n_epochs):

    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, X)
    loss.backward()
    optimizer.step()

    output_list.append(output)

scores = pixel_mse(output_list[-1], X).detach().cpu().numpy()
auc = get_auc(scores, label, resolution=101).round(3)
auc

In [ ]:
# Create DataFrames
df_X = pd.DataFrame(X.detach().cpu().numpy())
df_output = pd.DataFrame(output_list[-1].detach().cpu().numpy())

# Assign sensor IDs as index
df_X.index.name = "sensor_id"
df_output.index.name = "sensor_id"

# Melt to long format
df_X_long = df_X.reset_index().melt(id_vars=["sensor_id"], var_name="timestamp", value_name="X")
df_output_long = df_output.reset_index().melt(id_vars=["sensor_id"], var_name="timestamp", value_name="output")

# Merge both DataFrames
df_final = pd.merge(df_X_long, df_output_long, on=["sensor_id", "timestamp"])

# Convert timestamp to integer (assuming column names were originally numbers)
df_final["timestamp"] = df_final["timestamp"].astype(int)

print(f'{np.where(label)[0]}')
px.line(df_final, x='timestamp', y=['X','output'], animation_frame='sensor_id', width=1000, range_y=[-10,35]).show()


fig = px.line(y=[label*scores.max()*0.75, scores], width=1000, markers=True)  # Add markers
fig.update_traces(line=dict(width=0.5), marker={'size':5})  # Make line thin
fig.show()

px.line(df_final[df_final.sensor_id.isin(np.where(label)[0])], x='timestamp', y=['X','output'], animation_frame='sensor_id', width=1000, range_y=[-10,35]).show()

In [ ]:
G = fd.NNGraph(pd.DataFrame(data=dataset['pos'], columns=['easting','northing']), radius=15)
utils.plotly_signal(G, X[:,-1].cpu().numpy(), width=500, height=300)
utils.plotly_signal(G, label, width=500, height=300)

In [ ]:
model = models.GCNencoder([15,12,12])

In [ ]:
possible_classes = [models.GCN2MLP, models.AE]

In [ ]:
isinstance(model, tuple(possible_classes))

In [ ]:
pygsp.graphs.NNGraph(dataset['pos'])

In [ ]:
G = fd.NNGraph(pd.DataFrame(dataset['pos'], columns=['easting','northing']), radius=15)

In [ ]:
G.plot()

In [ ]:
from torch_geometric.utils import dense_to_sparse

In [ ]:
X.device

In [ ]:
next(model.parameters())

In [ ]:
dataset['edge_weight'].to(device)

In [ ]:
getattr(models, 'AE')([2, 2, 2, 2])